In [1]:
import csv
from dbfread import DBF 
import pandas as pd

In [2]:
#dbf_file = DBF('OD 2017/OD_2017_v1.dbf')

In [3]:
'''with open('OD_2017_original.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(dbf_file.field_names)
    for record in dbf_file:
        writer.writerow(list(record.values()))'''

"with open('OD_2017_original.csv', 'w', newline='') as csvfile:\n    writer = csv.writer(csvfile)\n    writer.writerow(dbf_file.field_names)\n    for record in dbf_file:\n        writer.writerow(list(record.values()))"

In [4]:
base = pd.read_csv('OD_2017_original.csv')

In [5]:
#A base original possui apenas variáveis numéricas dos tipos int e float. Algumas dessas variáveis numéricas são códigos descritos na documentação
base.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183092 entries, 0 to 183091
Data columns (total 128 columns):
 #    Column      Dtype  
---   ------      -----  
 0    ZONA        int64  
 1    MUNI_DOM    int64  
 2    CO_DOM_X    int64  
 3    CO_DOM_Y    int64  
 4    ID_DOM      int64  
 5    F_DOM       int64  
 6    FE_DOM      float64
 7    DOM         int64  
 8    CD_ENTRE    int64  
 9    DATA        int64  
 10   TIPO_DOM    int64  
 11   AGUA        int64  
 12   RUA_PAVI    int64  
 13   NO_MORAD    int64  
 14   TOT_FAM     int64  
 15   ID_FAM      int64  
 16   F_FAM       int64  
 17   FE_FAM      float64
 18   FAMILIA     int64  
 19   NO_MORAF    int64  
 20   CONDMORA    int64  
 21   QT_BANHO    float64
 22   QT_EMPRE    float64
 23   QT_AUTO     float64
 24   QT_MICRO    float64
 25   QT_LAVALOU  float64
 26   QT_GEL1     float64
 27   QT_GEL2     float64
 28   QT_FREEZ    float64
 29   QT_MLAVA    float64
 30   QT_DVD      float64
 31   QT_MICROON  float64
 32 

<>Removendo colunas desinteressantes para o trabalho<>
<>Os detalhes sobre cada coluna estão disponíveis no arquivo LAYOUT OD2017_v1.xlsx<>

In [7]:
colunas_removidas = ['ID_DOM', 'F_DOM', 'DOM', 'CD_ENTRE', 'DATA', 'AGUA', 'RUA_PAVI', 'ID_FAM', 'F_FAM', 'QT_BANHO', 'QT_EMPRE',
                     'QT_MICRO', 'QT_LAVALOU', 'QT_GEL1', 'QT_GEL2', 'QT_FREEZ', 'QT_MLAVA', 'QT_DVD', 'QT_MICROON', 'QT_SECAROU',
                     'NAO_DCL_IT', 'PONTO_BR', 'CD_RENFA', 'ID_PESS', 'F_PESS', 'PESSOA', 'MUNIESC', 'VINC1', 'VINC2',
                     'DIA_SEM', 'PAG_VIAG', 'TP_ESAUTO', 'VL_EST', 'VIA_BICI', 'TP_ESBICI']

colunas_categoricas = ['ZONA', 'MUNI_DOM', 'TIPO_DOM', 'CONDMORA', 'CRITERIOBR', 'SIT_FAM', 'SEXO', 'ESTUDA', 'GRAU_INS', 'CD_ATIVI', 'CO_REN_I', 'TIPO_ESC', 'TRAB1_RE', 'TRABEXT1', 'OCUP1',
                       'SETOR1', 'MUNITRA1', 'MUNITRA2', 'TRAB2_RE', 'TRABEXT2', 'OCUP2', 'SETOR2', 'ZONA_O', 'MUNI_O', 'ZONA_D', 'MUNI_D', 'ZONA_T1', 'MUNI_T1', 'ZONA_T2', 'MUNI_T2', 'ZONA_T3',
                       'MUNI_T3', 'MOTIVO_O', 'MOTIVO_D', 'MOT_SRES', 'SERVIR_O', 'SERVIR_D', 'MODO1', 'MODO2', 'MODO3', 'MODO4', 'MODOPRIN', 'TIPVG', 'PE_BICI']

df = base.drop(colunas_removidas, axis=1)
df[colunas_categoricas] = df[colunas_categoricas].astype('category'); 

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183092 entries, 0 to 183091
Data columns (total 93 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ZONA        183092 non-null  category
 1   MUNI_DOM    183092 non-null  category
 2   CO_DOM_X    183092 non-null  int64   
 3   CO_DOM_Y    183092 non-null  int64   
 4   FE_DOM      183092 non-null  float64 
 5   TIPO_DOM    183092 non-null  category
 6   NO_MORAD    183092 non-null  int64   
 7   TOT_FAM     183092 non-null  int64   
 8   FE_FAM      183092 non-null  float64 
 9   FAMILIA     183092 non-null  int64   
 10  NO_MORAF    183092 non-null  int64   
 11  CONDMORA    183092 non-null  category
 12  QT_AUTO     178930 non-null  float64 
 13  QT_MOTO     178898 non-null  float64 
 14  QT_BICICLE  178886 non-null  float64 
 15  CRITERIOBR  178886 non-null  category
 16  ANO_AUTO1   115248 non-null  float64 
 17  ANO_AUTO2   30966 non-null   float64 
 18  ANO_AUTO3   5446 non-nul